# **Generic**

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.signal import find_peaks

**Crop and Display Functions**

In [ ]:
def crop_barcode3(img):
    # Find contours in the binary image
    contours, _ = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour corresponds to the barcode
    largest_contour = max(contours, key=cv.contourArea)
    largest_contour_area = cv.contourArea(largest_contour)

    # Initialize variables to store the leftmost and rightmost contours
    leftmost_contour = None
    rightmost_contour = None
    x_min = float('inf')
    x_max = float('-inf')

    # Find the leftmost and rightmost contours
    for contour in contours:
        contour_area = cv.contourArea(contour)
        if contour_area >= 0.05 * largest_contour_area:  # Check if the contour area is significant
            x, _, w, _ = cv.boundingRect(contour)
            if x < x_min:
                x_min = x
                leftmost_contour = contour
            if x > x_max:  # Use x to get the far-right edge of the contour
                x_max = x
                rightmost_contour = contour

    x_min, _, _, _ = cv.boundingRect(leftmost_contour)
    x_max, _, w, _ = cv.boundingRect(rightmost_contour)

    # Get the vertical limits from the largest contour
    _, y, _, h = cv.boundingRect(largest_contour)

    # Crop the image so that only the barcode is visible
    cropped_img = img[y:y + h, x_min:x_max+w]

    # Draw all contours in red
    contour_img = cv.cvtColor(cv.bitwise_not(img), cv.COLOR_GRAY2BGR)  # Convert to BGR for colored drawing
    cv.drawContours(contour_img, contours, -1, (255, 0, 0), 4)  # Draw all contours in red

    return cropped_img, contour_img

def display_image(img, title):
    plt.imshow(img, cmap='gray')
    plt.title(title)
    # Draw border around the image
    plt.gca().add_patch(plt.Rectangle((0, 0), img.shape[1], img.shape[0], fill=None, edgecolor='red', linewidth=1))
    plt.axis('off')
    plt.show()

**Read Image**

In [ ]:
import cv2 as cv

# Prompt user to select the image number
print("Select the number of the image you want to load:")
print("1: 01 - lol easy.jpg")
print("2: 02 - still easy.jpg")
print("3: 03 - eda ya3am ew3a soba3ak mathazarsh.jpg")
print("4: 04 - fen el nadara.jpg")
print("5: 05 - meen taffa el nour!!!.jpg")
print("6: 06 - meen fata7 el nour 333eenaaayy.jpg")
print("7: 07 - mal7 w felfel.jpg")
print("8: 08 - compresso espresso.jpg")
print("9: 09 - e3del el soora ya3ammm.jpg")
print("10: 10 - wen el kontraastttt.jpg")
print("11: 11 - bayza 5ales di bsara7a.jpg")
print("12: Screenshot 2024-12-12 231530.png")

image_number = int(input("Enter the image number: "))

if image_number == 1:
    img = cv.imread('01 - lol easy.jpg')
elif image_number == 2:
    img = cv.imread('02 - still easy.jpg')
elif image_number == 3:
    img = cv.imread('03 - eda ya3am ew3a soba3ak mathazarsh.jpg')
elif image_number == 4:
    img = cv.imread('04 - fen el nadara.jpg')
elif image_number == 5:
    img = cv.imread('05 - meen taffa el nour!!!.jpg')
elif image_number == 6:
    img = cv.imread('06 - meen fata7 el nour 333eenaaayy.jpg')
elif image_number == 7:
    img = cv.imread('07 - mal7 w felfel.jpg')
elif image_number == 8:
    img = cv.imread('08 - compresso espresso.jpg')
elif image_number == 9:
    img = cv.imread('09 - e3del el soora ya3ammm.jpg')
elif image_number == 10:
    img = cv.imread('10 - wen el kontraastttt.jpg')
elif image_number == 11:
    img = cv.imread('11 - bayza 5ales di bsara7a.jpg')
elif image_number == 12:
    img = cv.imread('Samples/Screenshot 2024-12-12 231530.png')
else:
    print("Invalid image number. Please try again.")
    img = None

# Display the loaded image
if img is not None:
    display_image(cv.cvtColor(img, cv.COLOR_BGR2RGB), "Original Image")


#Utility Masks, might need for later

# Rectangle Mask
def generate_rectangle_mask(mask_dimensions, top_left, bottom_right):
    mask = np.zeros(mask_dimensions, dtype=np.uint8)
    return cv.rectangle(mask, top_left, bottom_right, 255, -1)


def generate_circle_mask(mask_dimensions, mask_radius):
    circle_mask = np.zeros(mask_dimensions)
    center_y = circle_mask.shape[0] // 2
    center_x = circle_mask.shape[1] // 2
    return cv.circle(circle_mask, (center_x, center_y), mask_radius, (255, 255, 255), -1).astype(np.uint8)



Select the number of the image you want to load:
1: 01 - lol easy.jpg
2: 02 - still easy.jpg
3: 03 - eda ya3am ew3a soba3ak mathazarsh.jpg
4: 04 - fen el nadara.jpg
5: 05 - meen taffa el nour!!!.jpg
6: 06 - meen fata7 el nour 333eenaaayy.jpg
7: 07 - mal7 w felfel.jpg
8: 08 - compresso espresso.jpg
9: 09 - e3del el soora ya3ammm.jpg
10: 10 - wen el kontraastttt.jpg
11: 11 - bayza 5ales di bsara7a.jpg
12: Screenshot 2024-12-12 231530.png
Enter the image number: 1


**Isolate Barcode (Test Case 3)**

In [ ]:
def isolate_barcode(image, threshold=10):

    # Convert image to float for better processing
    image = image.astype(np.float32)

    # Extract R, G, B channels
    B, G, R = cv.split(image)

    # Compute absolute differences between channels
    diff_rg = np.abs(R - G)
    diff_rb = np.abs(R - B)
    diff_gb = np.abs(G - B)

    # Create a mask where differences are below the threshold (grayscale condition)
    mask = (diff_rg < threshold) & (diff_rb < threshold) & (diff_gb < threshold)

    # Convert mask to binary (0 and 255)
    mask = mask.astype(np.uint8) * 255

    # Create a white canvas for the background
    white_canvas = np.ones_like(image, dtype=np.uint8) * 255  # White background

    # Apply mask to the original image to keep only grayscale pixels
    isolated_image = cv.bitwise_and(image, image, mask=mask)

    # Invert the mask to fill the background with white
    inverse_mask = cv.bitwise_not(mask)
    isolated_image += cv.bitwise_and(white_canvas, white_canvas, mask=inverse_mask)

    return isolated_image.astype(np.uint8)

# Isolate the barcode with white background
isolated_image = isolate_barcode(img, threshold=10)

display_image(cv.cvtColor(isolated_image, cv.COLOR_BGR2RGB), "Isolated Barcode (White Background)")


AttributeError: 'NoneType' object has no attribute 'astype'

**Convert to Greyscale**

In [ ]:
img = cv.cvtColor(isolated_image, cv.COLOR_BGR2GRAY)

# **Freq Domain Analaysis**


In [ ]:
view_index = 300


def apply_high_pass_filter(input_fft, cutoff_radius, apply_gaussian=False):
    high_pass_mask = ~generate_circle_mask(input_fft.shape, cutoff_radius)
    if apply_gaussian:
        high_pass_mask = cv.GaussianBlur(high_pass_mask, (21, 21), 0)

    high_pass_mask[input_fft.shape[0]//2, input_fft.shape[1]//2] = 255
    shifted_fft = np.fft.fftshift(input_fft)
    filtered_fft = np.multiply(shifted_fft, high_pass_mask)
    return process_shifted_fft(filtered_fft)

def apply_low_pass_filter(input_fft, cutoff_radius, apply_gaussian=False):
    low_pass_mask = generate_circle_mask(input_fft.shape, cutoff_radius)
    if apply_gaussian:
        low_pass_mask = cv.GaussianBlur(low_pass_mask, (21, 21), 0)
    shifted_fft = np.fft.fftshift(input_fft)
    filtered_fft = np.multiply(shifted_fft, low_pass_mask)
    return process_shifted_fft(filtered_fft)



def extract_intensity_values(image_data, selected_row=None, selected_col=None):
    img_rows, img_cols = image_data.shape
    if image_data is None:
        raise ValueError("Image not found or unable to load!")
    if selected_row is None and selected_col is None:
        selected_row = img_rows // 2
    elif selected_row is not None and (selected_row < 0 or selected_row >= img_rows):
        raise ValueError(f"Row doesn't exist")
    elif selected_col is not None and (selected_col < 0 or selected_col >= img_cols):
        raise ValueError(f"Column doesn't exist")
    elif selected_row is not None:
        intensity_values = image_data[selected_row, :]
        x_values = np.arange(img_cols)
        label_text = f"Row {selected_row}"
    else:
        intensity_values = image_data[:, selected_col]
        x_values = np.arange(img_rows)
        label_text = f"Column {selected_col}"

    plt.figure(figsize=(10, 4))
    plt.plot(x_values, intensity_values, label=label_text)
    plt.title('Time-Domain Representation of Pixel Intensities')
    plt.xlabel('Pixel Index')
    plt.ylabel('Intensity')
    plt.legend()
    plt.grid(True)
    plt.show()

    return intensity_values

intensity_values = extract_intensity_values(img, view_index, view_index)

def analyze_signal_peaks(signal_data, config_params):
    if 'min_peak_height' not in config_params or config_params['min_peak_height'] is None:
        raise ValueError("Height is needed")

    detected_peaks, _ = find_peaks(signal_data, height=config_params['min_peak_height'], distance=config_params['peak_distance'])

    if len(detected_peaks) == 0:
        print("Negative peak sequence")
        return {
            'are_peaks_uniform': False,
            'suggested_filter': None
        }

    peak_intervals = np.diff(detected_peaks)
    peaks_uniform = len(peak_intervals) > 0 and np.allclose(peak_intervals, peak_intervals[0], atol=config_params['distance_tolerance'])


    fft_result = np.fft.fft(signal_data)
    fft_magnitude_values = np.abs(fft_result)
    fft_freq_values = np.fft.fftfreq(len(signal_data), d=1/config_params['sampling_interval'])
    positive_freq_values = fft_freq_values[:len(signal_data)//2]
    positive_magnitude_values = fft_magnitude_values[:len(signal_data)//2]
    freq_peak_indices, _ = find_peaks(positive_magnitude_values, height=np.mean(positive_magnitude_values))
    dominant_frequencies = positive_freq_values[freq_peak_indices]
    dominant_magnitudes = positive_magnitude_values[freq_peak_indices]

    applied_filter_type = None
    if dominant_frequencies.size > 0:
        applied_filter_type = "Low-pass" if dominant_frequencies[0] * 10000 < 60 else "High-pass"
    else:
        applied_filter_type = "Low-pass"

    return {
        'are_peaks_uniform': peaks_uniform,
        'suggested_filter': applied_filter_type
    }

wave_analysis_config = {
    'min_peak_height': 145, 'peak_distance': 50,
    'distance_tolerance': 6, 'sampling_interval': 1.0
}

analysis_results = analyze_signal_peaks(intensity_values, wave_analysis_config)



def process_shifted_fft(shifted_fft_data):
    inverse_fft = np.fft.ifft2(np.fft.ifftshift(shifted_fft_data))
    magnitude_result = np.abs(inverse_fft)
    return magnitude_result.astype(np.uint16)


if analysis_results['are_peaks_uniform']:
    fft_image = np.fft.fft2(img)
    if analysis_results['suggested_filter'] == "High-pass":
        processed_image = apply_high_pass_filter(fft_image, 20, apply_gaussian=False)
    elif analysis_results['suggested_filter'] == "Low-pass":
        processed_image = apply_low_pass_filter(fft_image, 125, apply_gaussian=True)

    processed_image = np.abs(processed_image)
    processed_image = np.uint8(255 * (processed_image / np.max(processed_image)))
    if len(processed_image.shape) > 2:
        processed_image = cv.cvtColor(processed_image, cv.COLOR_BGR2GRAY)
    freq_fixed_img = processed_image[:, 50:-50]
else:
    freq_fixed_img = img


In [ ]:


if analysis_results['are_peaks_uniform']:
    fft_image = np.fft.fft2(img)
    if analysis_results['suggested_filter'] == "High-pass":
        processed_image = apply_high_pass_filter(fft_image, 20, apply_gaussian=False)
    elif analysis_results['suggested_filter'] == "Low-pass":
        processed_image = apply_low_pass_filter(fft_image, 125, apply_gaussian=True)

    processed_image = np.abs(processed_image)
    processed_image = np.uint8(255 * (processed_image / np.max(processed_image)))
    if len(processed_image.shape) > 2:
        processed_image = cv.cvtColor(processed_image, cv.COLOR_BGR2GRAY)
    freq_fixed_img = processed_image[:, 50:-50]
else:
    freq_fixed_img = img


**Apply Median Filter**

In [ ]:

##kernel = np.ones((3,1),np.float32)/2.5
##noise_fixed_img = cv.filter2D(freq_fixed_img,-1,kernel)
##display_image(noise_fixed_img, 'Median Blurred Image')
""""
median_blur = cv.medianBlur(freq_fixed_img, 9)
display_image(median_blur, 'Median Blurred Image')
"""

##vertical_median_blur = cv.medianBlur(cv.blur(freq_fixed_img, (5, 5)), 3)
##display_image(vertical_median_blur, 'Vertical Median Blurred Image')

# Apply Morph Open to remove noise
kernel = np.ones((3, 3), np.uint8)
opening = cv.morphologyEx(freq_fixed_img, cv.MORPH_CLOSE, kernel, iterations=1)
display_image(opening, 'Morphological Opening')




**Thresholding**

In [ ]:
_, binary_img = cv.threshold(opening, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)
display_image(binary_img, 'Binary Image')


**Fix Rotation**

In [ ]:
def detect_and_rotate(image):

    # Find contours
    contours, _ = cv.findContours(image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area
    all_contours = []
    for contour in contours:
        if cv.contourArea(contour) > 100:  # Only consider larger contours that could be part of the barcode
            all_contours.append(contour)

    # Combine all contours to analyze the overall bounding rectangle
    combined_contour = np.vstack(all_contours)

    # Get the minimum area rectangle around the combined contour
    rect = cv.minAreaRect(combined_contour)
    angle = rect[2]

    # If the angle is not near zero, the barcode is rotated
    if abs(angle) > 1:

        # Adjust the rotation angle
        if angle < -45:
            angle = 90 + angle
        elif angle > 45:
            angle = angle - 90

        # Compute the rotation matrix
        # Get the center of the bounding rectangle
        center = (int(rect[0][0]), int(rect[0][1]))

        # Get the rotation matrix
        rotation_matrix = cv.getRotationMatrix2D(center, angle, 1.0)

        # Get the dimensions of the image
        rows, cols = image.shape[:2]

        # Rotate the image using warpAffine
        rotated_image = cv.warpAffine(image, rotation_matrix, (cols, rows), flags=cv.INTER_CUBIC, borderMode=cv.BORDER_REPLICATE)

        return rotated_image

    else:
        return image  # No rotation needed





# Check and rotate the image if needed
rotated_img = detect_and_rotate(binary_img)

# Morph close to fill gaps
kernel = np.ones((7, 1), np.uint8)
closed_img = cv.morphologyEx(rotated_img, cv.MORPH_CLOSE, kernel, iterations=1)
display_image(closed_img, 'Closed Image')

display_image(cv.bitwise_not(rotated_img), 'Rotated Image')



**Morph Open**

In [ ]:
vertical_kernel = np.ones((60, 1), np.uint8)
opened_img = cv.morphologyEx(closed_img, cv.MORPH_OPEN, vertical_kernel, iterations=1)
display_image(opened_img, 'Opened Image')

**Crop, Close, Dilate**

In [ ]:
cropped_img, contour_img = crop_barcode3(opened_img)
display_image(cv.bitwise_not(cropped_img), 'Cropped Image')

# Close the cropped image
kernel = np.ones((100, 1), np.uint8)
closed_img = cv.morphologyEx(cropped_img, cv.MORPH_CLOSE, kernel, iterations=3)
display_image(cv.bitwise_not(closed_img), 'Closed Image')


**Decode**

In [ ]:

# 0 means narrow, 1 means wide
NARROW = "0"
WIDE = "1"
code11_widths = {
    "00110": "Stop/Start",
    "10001": "1",
    "01001": "2",
    "11000": "3",
    "00101": "4",
    "10100": "5",
    "01100": "6",
    "00011": "7",
    "10010": "8",
    "10000": "9",
    "00001": "0",
    "00100": "-",
}

# Threshold the cropped image
mean = cv.bitwise_not(closed_img).mean(axis=0)  # Column-wise mean
mean = np.where(mean <= 127, 1, 0)  # Black or White

print("Mean Array:", mean)  # Check the binary output

# Convert to string of pixels in order to loop over it
pixels = list(''.join(mean.astype(np.uint8).astype(str)))

# Remove up to 15 zeros from the start
trim_count = 0
while len(pixels) > 0 and trim_count < 150 and pixels[0] == "0":
    pixels.pop(0)
    trim_count += 1

# Remove up to 15 zeros from the end
trim_count = 0
while len(pixels) > 0 and trim_count < 150 and pixels[-1] == "0":
    pixels.pop()
    trim_count += 1

# Print after trimming
print("Trimmed Mean Array:", pixels)

# Convert list back to string
pixels = ''.join(pixels)

# Need to figure out how many pixels represent a narrow bar
narrow_bar_size = 0
for pixel in pixels:
    if pixel == "1":
        narrow_bar_size += 1
    else:
        break
print("Narrow Bar Size:", narrow_bar_size)

wide_bar_size = narrow_bar_size * 2

digits = []
pixel_index = 0
current_digit_widths = ""
skip_next = False

while pixel_index < len(pixels):

    if skip_next:
        pixel_index += narrow_bar_size
        skip_next = False
        continue

    count = 1
    try:
        while pixels[pixel_index] == pixels[pixel_index + 1]:
            count += 1
            pixel_index += 1
    except:
        pass
    pixel_index += 1

    # Determine if the bar width is narrow or wide
    if 3 <= count <= 5:
        current_digit_widths += NARROW
    elif 7 <= count <= 9:
        current_digit_widths += WIDE

    if current_digit_widths in code11_widths:
        digits.append(code11_widths[current_digit_widths])
        current_digit_widths = ""
        skip_next = True  # Next iteration will be a separator, so skip it

print(digits)


if (image_number == 1):
    print("Expected Result: 1234567890-")
elif (image_number == 2):
    print("Expected Result: 104-116-116")
elif (image_number == 3):
    print("Expected Result: 112-115-58-")
elif (image_number == 4):
    print("Expected Result: -47-47-121-")
elif (image_number == 5):
    print("Expected Result: 111-117-116")
elif (image_number == 6):
    print("Expected Result: -117-46-98-")
elif (image_number == 7):
    print("Expected Result: 101-47-100-")
elif (image_number == 8):
    print("Expected Result: 113-119-52-")
elif (image_number == 9):
    print("Expected Result: 119-57-119-")
elif (image_number == 10):
    print("Expected Result: 103-120-99-")
elif (image_number == 11):
    print("Expected Result: 113-47-35-35")